### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
from ads.jobs import Job
from ads.jobs import DataScienceJob, ScriptRuntime
import ads 
ads.set_auth('resource_principal')

In [ ]:
infrastructure = (
    DataScienceJob()
    .with_shape_name("VM.Standard2.24")
    .with_block_storage_size(50)
    .with_log_group_id("")
    .with_log_id("")
)

In [ ]:
runtime = (
    ScriptRuntime()
    .with_source("./training_vgg16.py")
    .with_service_conda("tensorflow26_p37_gpu_v3")
)

In [ ]:
job = Job(name='vgg16-training').with_infrastructure(infrastructure).with_runtime(runtime)

In [ ]:
job.to_yaml('training-config.yaml')

In [ ]:
job.create()

In [ ]:
job_run = job.run()

In [ ]:
job_run.watch()